In [ ]:
from CDM_DataSetMaker_ops import get_conn
from RETAIN_Dataset_Final import convert_transaction_to_seq

conn = get_conn('./DB_connection.txt')
#T_seq = convert_transaction_to_seq(conn, 'condition_concept_id', 'HF_T_dx.pkl')
#C_seq = convert_transaction_to_seq(conn, 'condition_concept_id', 'HF_C_dx.pkl')

In [1]:
from CDM_DataSetMaker_ops import loadingFiles

T_seq = loadingFiles('./', 'HF_T_dx.pkl')
C_seq = loadingFiles('./', 'HF_C_dx.pkl')

len(T_seq), len(C_seq)

Loading at.. ./HF_T_dx.pkl
Loading at.. ./HF_C_dx.pkl


(22548, 85011)

In [2]:
T_sp = T_seq[:10]
C_sp = C_seq[:20]

In [3]:
from RETAIN_Dataset_Final import truncated_data, code_to_id

HF_T_cut = truncated_data(T_sp, cut_num=2)
HF_C_cut = truncated_data(C_sp, cut_num=2)
HF_code_dict, max_visit_size = code_to_id(HF_T_cut+HF_C_cut)

code_size:  261


In [4]:
from RETAIN_Dataset_Final import RETAIN_datasets

datasets = RETAIN_datasets(HF_T_cut, HF_C_cut, HF_code_dict, max_visit_size)

In [9]:
datasets.train.next_batch(10)

(5,) 
 (5, 30, 261)


ValueError: all the input arrays must have same number of dimensions

In [5]:
import numpy as np

a = np.arange(6).reshape([2,3]).tolist()
b = np.arange(21).reshape([7,3]).tolist()

print(b*0)

[]


In [5]:
datasets.train.next_batch(10)

TypeError: _shuffle() takes 2 positional arguments but 4 were given

In [6]:
a.shape, b.shape, c.shape

NameError: name 'a' is not defined

In [10]:
import numpy as np

np.concatenate((a[0], b[0]))[0]

array([<30x187 sparse matrix of type '<class 'numpy.float32'>'
	with 48 stored elements in Compressed Sparse Row format>,
       list([[1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0], [1, 0]]),
       13], dtype=object)

In [16]:
a[1]

array([list([[77074, 4001450], [4001450], [320128, 77074, 4001450], [4001450], [4001450, 439418], [77074, 4001450], [4001450], [4001450], [4001450], [4001450], [4001450], [4001450], [4001450], [4001450], [77074], [4001450], [4001450], [4001450], [320128, 77074], [4001450], [4001450], [4001450], [77074], [4001450, 320128, 77074], [4001450], [4001450], [4001450], [4001450], [4001450], [320128, 77074]])],
      dtype=object)

In [17]:
a[2]

array([list([[320128, 4193704, 381035, 432867, 42709799], [42709799, 320128, 4193704, 381035, 432867], [320128, 4193704, 381035, 432867, 42709799], [320128, 4193704, 381035, 432867, 42709799], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [4193704, 381035, 432867, 42709799, 320128], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [42709799, 320128, 4193704, 381035, 432867], [255841, 433596, 4151422, 254761, 439418, 320128], [320128, 4193704, 4291025, 432867, 4031019], [320128, 4193704, 4291025, 436147], [320128, 4193704, 4291025, 4225273], [320128, 4193704, 4291025, 4225273], [320128, 4193704, 4291025, 321052, 381035]])],
      dtype=object)

In [47]:
import tensorflow as tf

In [6]:
batch_size = 10
time_size = max_visit_size
code_size = len(HF_code_dict)
label_size = 2
lr_init = 0.0001
decay_step = 2000
decay_rate = 0.9
training_step = 5000
printby = 100

hidden_size_alpha = 128
hidden_size_beta = 128

In [ ]:
def sequence_masking(data, visit_times):
    masking = tf.tile(tf.reshape(tf.sequence_mask(visit_times, data.shape[1]), shape=[-1,data.shape[1],1]), [1,1,data.shape[2]])
    return tf.where(masking, data, tf.zeros_like(data))

inputs = tf.placeholder(tf.float32, shape=[None, time_size, code_size])
labels = tf.placeholder(tf.float32, shape=[None, time_size, label_size])
visit_times = tf.placeholder(tf.float32, shape=[None])
global_step = tf.Variable(0, trainable=False, dtype=tf.int32)
lr = tf.train.exponential_decay(lr_init, global_step, decay_step, decay_rate, staircase=True)

##embedding

##alpha
W_alpha = tf.Variable(tf.random_normal(shape=[hidden_size_alpha, 1], name='W_alpha'))
b_alpha = tf.Variable(tf.random_normal(shape=[], name='b_alpha'))
with tf.variable_scope('alpha'):
    cells_alpha = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(hidden_size_alpha)])
    outputs_alpha, states_alpha = tf.nn.dynamic_rnn(cells_alpha, inputs, visit_times, dtype=tf.float32)
    reshaped_outputs_alpha = tf.reshape(outputs_alpha, shape=[-1, hidden_size_alpha])
    matmuled_alpha = tf.matmul(reshaped_outputs_alpha, W_alpha) + tf.expand_dims(b_alpha, 0)
    reshaped_matmuled_alpha = tf.reshape(matmuled_alpha, shape=[-1, time_size, 1])
    logits_alpha = tf.nn.softmax(reshaped_matmuled_alpha, 1)
    
##Beta
W_beta = tf.Variable(tf.random_normal(shape=[hidden_size_beta, code_size]))
b_beta = tf.Variable(tf.random_normal(shape=[code_size]))
with tf.variable_scope('beta'):
    cells_beta = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.LSTMCell(hidden_size_beta)])
    outputs_beta, states_beta = tf.nn.dynamic_rnn(cells_beta, inputs, visit_times, dtype=tf.float32)
    reshaped_outputs_beta = tf.reshape(outputs_beta, shape=[-1, hidden_size_beta])
    matmuled_ouputs_beta = tf.matmul(reshaped_outputs_beta, W_beta) + tf.expand_dims(b_beta, 0)
    reshaped_matmuled_beta = tf.reshape(matmuled_ouputs_beta, shape=[-1, time_size, code_size])
    logits_beta = tf.nn.tanh(reshaped_matmuled_beta)
    
##unifiy
W_s = tf.Variable(tf.random_normal(shape=[code_size, label_size]))
b_s = tf.Variable(tf.random_normal(shape=[label_size]))
unified_context = logits_alpha*logits_beta*inputs
context_vec = []
for b in range(batch_size):
    pid_seq = unified_context[b]
    for t in range(time_size):
        context_vec.append(tf.reduce_sum(pid_seq[:t+1, :], axis=0))
reshaped_context_vec = tf.reshape(context_vec, shape=[-1, code_size])
mlp_context_vec = tf.matmul(reshaped_context_vec, W_s) + b_s
logits = tf.nn.softmax(tf.reshape(mlp_context_vec, shape=[-1, time_size, label_size]))
masked_logits = sequence_masking(logits, visit_times)
masked_labels = sequence_masking(labels, visit_times)

##Loss function
loss_per_times = tf.reduce_sum(masked_labels*tf.log(masked_logits+1e-10)+(1-masked_labels)*tf.log(1-masked_logits+1e-10), axis=-1)
loss_per_pid = tf.reduce_sum(loss_per_times, axis=-1)*(1/visit_times)
last_loss = -tf.reduce_sum(loss_per_pid)*(1/batch_size)
optimizer = tf.train.AdamOptimizer(learning_rate=lr).minimize(last_loss, global_step=global_step)

##accuracy
argmax_logits = tf.argmax(tf.reshape(masked_logits, shape=[-1,label_size]), axis=1)
argmax_labels = tf.argmax(tf.reshape(masked_labels, shape=[-1,label_size]), axis=1)
compared = tf.cast(tf.equal(argmax_logits, argmax_labels), tf.float32)
accuracy = tf.reduce_mean(compared, axis=0)

In [ ]:
from tqdm import trange
with tf.Session() as sess:
    tf.train.start_queue_runners(sess=sess)
    tf.global_variables_initializer().run(session=sess)

    for step in trange(training_step):
        batch_inputs, batch_labels, batch_visit_times = datasets.next_batch(batch_size)
        feed_dict = {inputs: batch_inputs, 
                     labels: batch_labels, 
                     visit_times: batch_visit_times}
        _, g_step, loss, acc, lr_ = sess.run([optimizer, global_step, last_loss, accuracy, lr], feed_dict=feed_dict)
        W_alpha_, b_alpha_, W_beta_, b_beta_ = sess.run([W_alpha, b_alpha, W_beta, b_beta], feed_dict=feed_dict)
        
        if step % printby == 0:
            print('step: {} \t||g_step: {} \t||acc: {:.2f}% \t||loss: {:.8f} \t|| lr: {:.8f}'.format(step, g_step, acc*100, loss, lr_))
            print('||W_alpha|| \n{}\n \n||b_alpha|| \n{}\n \n||W_beta|| \n{}\n \n||b_beta|| \n{}'.format(W_alpha_, b_alpha_, W_beta_, b_beta_))

  0%|          | 1/5000 [00:03<4:41:43,  3.38s/it]

step: 0 	||g_step: 1 	||acc: 59.67% 	||loss: 1.48455703 	|| lr: 0.00010000
||W_alpha|| 
[[-1.0518664 ]
 [-0.9711591 ]
 [-1.2181786 ]
 [ 0.6566554 ]
 [-2.2886965 ]
 [-0.5072838 ]
 [ 0.02993489]
 [-1.3562381 ]
 [-1.5627445 ]
 [-0.02806907]
 [ 1.5066665 ]
 [ 0.22895621]
 [ 1.4107505 ]
 [ 1.4654175 ]
 [ 0.13107178]
 [ 0.8714891 ]
 [ 0.0708207 ]
 [ 1.1766695 ]
 [-0.8058364 ]
 [-1.1004428 ]
 [-2.0545974 ]
 [ 2.4650793 ]
 [-1.2931994 ]
 [-1.4240856 ]
 [-0.66167784]
 [-0.4067322 ]
 [-0.01419307]
 [-1.2785504 ]
 [ 0.4235855 ]
 [-0.0143417 ]
 [-0.44829413]
 [ 0.4462092 ]
 [-0.13423721]
 [ 0.45018083]
 [ 0.5150524 ]
 [ 0.95240724]
 [ 0.7137355 ]
 [ 1.8591989 ]
 [ 0.5431599 ]
 [-1.1461482 ]
 [-0.8313645 ]
 [-0.62764406]
 [-0.6551012 ]
 [ 0.348292  ]
 [-0.07818298]
 [ 0.47897318]
 [-0.5449825 ]
 [ 0.08252742]
 [ 1.6324838 ]
 [ 0.03590515]
 [-0.01744051]
 [-0.36247742]
 [ 0.18344584]
 [ 0.3701772 ]
 [-0.3754135 ]
 [ 0.5102028 ]
 [ 1.1047804 ]
 [-0.21653591]
 [-0.11529252]
 [ 1.1698632 ]
 [ 1.0712034

  2%|▏         | 101/5000 [02:46<2:13:00,  1.63s/it]

step: 100 	||g_step: 101 	||acc: 70.00% 	||loss: 1.27241290 	|| lr: 0.00010000
||W_alpha|| 
[[-1.0505508 ]
 [-0.97134894]
 [-1.2162936 ]
 [ 0.65639824]
 [-2.2902644 ]
 [-0.5094594 ]
 [ 0.03070374]
 [-1.3590001 ]
 [-1.5636511 ]
 [-0.02953732]
 [ 1.5068997 ]
 [ 0.22837272]
 [ 1.4112389 ]
 [ 1.4665136 ]
 [ 0.13096273]
 [ 0.8695322 ]
 [ 0.06997633]
 [ 1.1769035 ]
 [-0.8053299 ]
 [-1.1012026 ]
 [-2.054512  ]
 [ 2.4628298 ]
 [-1.2955744 ]
 [-1.4245037 ]
 [-0.66256833]
 [-0.40825954]
 [-0.01660764]
 [-1.2773238 ]
 [ 0.42415002]
 [-0.01422694]
 [-0.4459303 ]
 [ 0.44776928]
 [-0.1310546 ]
 [ 0.44901603]
 [ 0.5137246 ]
 [ 0.954261  ]
 [ 0.7118353 ]
 [ 1.8580313 ]
 [ 0.5441232 ]
 [-1.14692   ]
 [-0.83149254]
 [-0.6257487 ]
 [-0.6563099 ]
 [ 0.3476611 ]
 [-0.07584663]
 [ 0.47831133]
 [-0.54432863]
 [ 0.08185599]
 [ 1.6330441 ]
 [ 0.03850923]
 [-0.0159401 ]
 [-0.36337858]
 [ 0.18331143]
 [ 0.36933124]
 [-0.37436724]
 [ 0.5107477 ]
 [ 1.1054024 ]
 [-0.21661082]
 [-0.11582398]
 [ 1.1713902 ]
 [ 1.070

  4%|▍         | 201/5000 [05:26<2:06:30,  1.58s/it]

...

 44%|████▍     | 2201/5000 [1:03:42<1:19:00,  1.69s/it]

step: 2200 	||g_step: 2201 	||acc: 80.67% 	||loss: 0.92833918 	|| lr: 0.00009000
||W_alpha|| 
[[-1.0728624 ]
 [-0.9855299 ]
 [-1.231874  ]
 [ 0.6658208 ]
 [-2.303879  ]
 [-0.5253511 ]
 [ 0.02945268]
 [-1.3782998 ]
 [-1.5798801 ]
 [-0.03147427]
 [ 1.5421747 ]
 [ 0.2512326 ]
 [ 1.4348023 ]
 [ 1.4871422 ]
 [ 0.14570425]
 [ 0.8814098 ]
 [ 0.08927733]
 [ 1.1917518 ]
 [-0.82658166]
 [-1.1188127 ]
 [-2.0659702 ]
 [ 2.481212  ]
 [-1.3146354 ]
 [-1.4369301 ]
 [-0.693988  ]
 [-0.4267222 ]
 [-0.02822842]
 [-1.3002164 ]
 [ 0.43883842]
 [-0.01667147]
 [-0.46661028]
 [ 0.46451756]
 [-0.147594  ]
 [ 0.45363063]
 [ 0.5322901 ]
 [ 0.97640187]
 [ 0.7177005 ]
 [ 1.874935  ]
 [ 0.56079024]
 [-1.166355  ]
 [-0.85136116]
 [-0.648006  ]
 [-0.6760224 ]
 [ 0.37766907]
 [-0.08175103]
 [ 0.49550927]
 [-0.56998134]
 [ 0.09803762]
 [ 1.6433747 ]
 [ 0.04445343]
 [-0.01141578]
 [-0.38885054]
 [ 0.18467268]
 [ 0.38895455]
 [-0.38710803]
 [ 0.52728903]
 [ 1.1189209 ]
 [-0.2308214 ]
 [-0.14018844]
 [ 1.1928936 ]
 [ 1.0

 46%|████▌     | 2301/5000 [1:06:25<1:12:15,  1.61s/it]

step: 2300 	||g_step: 2301 	||acc: 98.00% 	||loss: 0.56400198 	|| lr: 0.00009000
||W_alpha|| 
[[-1.0767798 ]
 [-0.98676157]
 [-1.23443   ]
 [ 0.6676768 ]
 [-2.3067956 ]
 [-0.5279264 ]
 [ 0.0273737 ]
 [-1.3800439 ]
 [-1.5814984 ]
 [-0.03197803]
 [ 1.5451387 ]
 [ 0.25329217]
 [ 1.43768   ]
 [ 1.49013   ]
 [ 0.145876  ]
 [ 0.88343984]
 [ 0.08928026]
 [ 1.1923205 ]
 [-0.8275616 ]
 [-1.1205565 ]
 [-2.0677793 ]
 [ 2.4842222 ]
 [-1.3164163 ]
 [-1.4391361 ]
 [-0.69703406]
 [-0.42968962]
 [-0.02958498]
 [-1.3020574 ]
 [ 0.4385522 ]
 [-0.01805402]
 [-0.46902943]
 [ 0.46533173]
 [-0.14992774]
 [ 0.45459813]
 [ 0.5348697 ]
 [ 0.97759074]
 [ 0.71738935]
 [ 1.8776265 ]
 [ 0.56116503]
 [-1.168519  ]
 [-0.852785  ]
 [-0.65178645]
 [-0.677271  ]
 [ 0.37851867]
 [-0.08539351]
 [ 0.49918902]
 [-0.573673  ]
 [ 0.10016941]
 [ 1.6450497 ]
 [ 0.04433347]
 [-0.01207813]
 [-0.3903527 ]
 [ 0.18429975]
 [ 0.39264727]
 [-0.38641137]
 [ 0.5289091 ]
 [ 1.1202384 ]
 [-0.23313294]
 [-0.14111753]
 [ 1.1950507 ]
 [ 1.1

 46%|████▌     | 2310/5000 [1:06:42<1:23:19,  1.86s/it]